# Peak detection

What do we consider to be a peak? Visually, it's the top of the hill, but for a computer
we need to find a metic. Thus, let's define the peak as local maxima which respects a
predefined list of criteria.
[scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html)
is perfect for the job as it looks for local maxima which meets a list of criteria.

## Criteria

### Height

The height criterium limits the peak detection to values above a certain `y` value.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import find_peaks, peak_prominences


# create a signal
x = np.linspace(0, 6 * np.pi, 1000)
x = np.sin(x) + 0.6 * np.sin(2.6 * x)

# find peaks with a height of at least 1
peaks, _ = find_peaks(x, height=1)

# plot
f, ax = plt.subplots(1, 1, layout="constrained")
ax.plot(x)
ax.axhline(y=1, color="teal", linestyle="--")
ax.plot(peaks, x[peaks], "x", markersize=10)
plt.show()

### Prominence

`scipy` defines the prominence as how much a peak stands out from the surrounding 
baseline of the signal and is defined as the vertical distance between the peak and its lowest contour line.

In [ ]:
# create a signal
x = np.linspace(0, 6 * np.pi, 1000)
x = np.sin(x) + 0.6 * np.sin(2.6 * x)

# find peaks and measure prominence
peaks, _ = find_peaks(x)
prominences = peak_prominences(x, peaks)[0]

# plot
contour_heights = x[peaks] - prominences
plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.vlines(x=peaks, ymin=contour_heights, ymax=x[peaks])
plt.show()

## Respiration signal

The respiration signal is:
- (1) noisy as all physiological signal which means that we need to filter it before we look for a peak
- (2) contains wide peaks with 'flat' tops, which means that depending on the filter applied, the local maxima might slightly move left or right

### Local maxima for different filters

Let's have a look in the position of the local maxima found by 
[scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html) for
different filters of the same 4 seconds of data.

In [ ]:
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from mne import set_log_level
from mne.io import read_raw_fif
from mne_lsl.stream._filters import create_filter
from scipy.signal import sosfilt

import resp_audio_sleep


set_log_level("WARNING")
# load data
root = Path(resp_audio_sleep.__file__).parent.parent / "data"
fname = root / "isochronous-raw.fif"
raw = read_raw_fif(fname, preload=False).pick("AUX7").crop(20, 40).load_data()
data = raw.get_data().T  # transpose to match what MNE-LSL prepared the filters for
# IIR filters with initial response estimated as a step response steady-state with
# scipy.signal.sosfilt_zi
filter1 = create_filter(raw.info["sfreq"], None, 10, iir_params=None)
filter1["zi"] = np.mean(data, axis=0) * filter1["zi_unit"]
filter2 = create_filter(raw.info["sfreq"], None, 40, iir_params=None)
filter2["zi"] = np.mean(data, axis=0) * filter2["zi_unit"]
data_filter1, _ = sosfilt(filter1["sos"], data, axis=0, zi=filter1["zi"])
data_filter2, _ = sosfilt(filter2["sos"], data, axis=0, zi=filter2["zi"])

# plot
f, ax = plt.subplots(1, 1, layout="constrained")
ax.plot(raw.times, data.squeeze(), color="blue", label="raw")
ax.plot(raw.times, data_filter1.squeeze(), color="teal", label="10 Hz low-pass")
ax.plot(raw.times, data_filter2.squeeze(), color="orange", label="40 Hz low-pass")
ax.legend()
plt.show()

The 'strange' start behavior is due to the initial conditions being set to the min of
the signal. To ignore this section, let's look for peaks which are above the mean
value of the signal.

In [ ]:
from scipy.signal import find_peaks


# find peaks with constrains
peaks_raw, _ = find_peaks(data.squeeze(), height=np.mean(data.squeeze()), prominence=50)
peaks_filter1, _ = find_peaks(
    data_filter1.squeeze(), height=np.mean(data_filter1.squeeze()), prominence=10
)
peaks_filter2, _ = find_peaks(
    data_filter2.squeeze(), height=np.mean(data_filter2.squeeze()), prominence=10
)

# plot
f, ax = plt.subplots(1, 1, layout="constrained")
ax.plot(raw.times, data.squeeze(), color="blue", label="raw")
ax.plot(raw.times, data_filter1.squeeze(), color="teal", label="10 Hz low-pass")
ax.plot(raw.times, data_filter2.squeeze(), color="orange", label="40 Hz low-pass")
for peak in peaks_raw:
    ax.axvline(raw.times[peak], color="blue", linestyle="--")
for peak in peaks_filter1:
    ax.axvline(raw.times[peak], color="teal", linestyle="--")
for peak in peaks_filter2:
    ax.axvline(raw.times[peak], color="orange", linestyle="--")
ax.legend()
plt.show()

Now we can start to see the issue, due to the peak shape, the local maxima is moving a bit based on the processing applied. Let's zoom on the last peak.

In [ ]:
f, ax = plt.subplots(1, 1, layout="constrained")
ax.plot(raw.times, data.squeeze(), color="blue", label="raw")
ax.plot(raw.times, data_filter1.squeeze(), color="teal", label="10 Hz low-pass")
ax.plot(raw.times, data_filter2.squeeze(), color="orange", label="40 Hz low-pass")
for peak in peaks_raw:
    ax.axvline(raw.times[peak], color="blue", linestyle="--")
for peak in peaks_filter1:
    ax.axvline(raw.times[peak], color="teal", linestyle="--")
for peak in peaks_filter2:
    ax.axvline(raw.times[peak], color="orange", linestyle="--")
ax.legend()
# select the last second
ax.set_xlim(raw.times[-int(1 * raw.info["sfreq"])], raw.times[-1])
plt.show()

Let's have a look at the distance between the 10 Hz low-pass filtered local maxima and the
40 Hz low-pass filtered local maxima.

In [ ]:
delay = 1000 * (peaks_filter1[-1] - peaks_filter2[-1]) / raw.info["sfreq"]
print(  # noqa: T201
    f"Distance between 10 Hz and 40 Hz low-pass signal local maximas: {delay:.1f} ms."
)

## Filters chosen for the online detector

For the online detector, I chose to use 3 filters:
- (1) IIR causal butter order 4 notch @ 50 Hz
- (2) IIR causal butter order 4 notch @ 100
- (3) IIR causal butter order 4 low-pass @ 20 Hz

This combination does not distort the phase of the signal too much and thus does not
shift the peak compared to its real location.

An acausal FIR filter works (no noticeable boundary effect) but is (1) slower and (2) 
exacerbate the offline replication problem. Thus, I kept the IIR instead.


In [ ]:
raw.crop(10, 20)
raw_filtered = raw.copy()
raw_filtered.notch_filter(50, picks="AUX7", method="iir", phase="forward")
raw_filtered.notch_filter(100, picks="AUX7", method="iir", phase="forward")
raw_filtered.filter(None, 20, picks="AUX7", method="iir", phase="forward")
raw.crop(5, None)
raw_filtered.crop(5, None)  # time for the filter without initial state to settle


f, ax = plt.subplots(1, 1, layout="constrained")
ax.plot(raw.times, raw.get_data().squeeze(), color="gray", label="raw")
ax.plot(raw.times, raw_filtered.get_data().squeeze(), color="teal", label="filtered")
ax.legend()
plt.show()

# Online detection and triggering logic

Let's now cover the entire online detection logic.

## Detector

### Circular buffer

The detector has a 4 seconds circular buffer that sees new data coming in and old data
coming out. When new samples are available, they are retrieved, filtered, and added to
the buffer.

<div style="text-align:center"><img src="https://mne.tools/mne-lsl/stable/_images/circular-buffer-light.png" /></div>

In practice, the circular buffer is implemented as a `numpy` array ordered from left to
right:

- on the left, the old samples
- on the right, the new samples

When `N` new samples are added to the buffer, we roll the buffer by N thus moving the N
first sample to the end, and then we replace those last N samples with the N new samples
retrieved.


In [ ]:
import numpy as np


# buffer of 10 samples: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
buffer = np.arange(10)
print("Buffer:\t\t\t", buffer)  # noqa: T201
new_samples = np.array([10, 11, 12])
buffer = np.roll(buffer, -new_samples.size)
print("Buffer rolled:\t\t", buffer)  # noqa: T201
buffer[-new_samples.size :] = new_samples
print("Buffer updated:\t\t", buffer)  # noqa: T201

### Acquiring and filtering new samples

In an online loop, the acquisition of new samples and the update of the circular buffer
is coded as follows:

In [ ]:
# Pseudo-code, do not run this cell

# construct filter
from scipy.signal import sosfilt, sosfilt_zi

filter = create_filter(...)  # create second order sections (sos)  # noqa: A001
filter["zi_unit"] = sosfilt_zi(filter["sos"])  # create initial conditions
filter["zi"] = (
    None  # initial conditions for the filter that will be updated in the first iteration of the loop  # noqa: E501
)

# create buffer
import numpy as np  # noqa: E402

buffer = np.zeros(...)

# online acquisition loop
while True:
    # query the network for new samples, we might get some or we might get none
    new_samples, new_samples_timestamps = inlet.pull_chunk()  # noqa: F821
    if new_samples_timestamps.size == 0:
        # no new samples, thus we 'continue', i.e. we go back to the beginning of the
        # loop and query the network again (without delay) for new samples
        continue

    # process the new samples, in this case apply the filter. We need to know the filter
    # state 'zi', which is either the state of the N-1 iteration, or the initial state
    # estimated as the steady-state response of the filter to a step input.
    if filter["zi"] is None:
        # first iteration, estimate using a step input equal to the mean of the signal
        # which is realistic for EEG signal subject to an important DC offset.
        filter["zi"] = np.mean(new_samples) * filter["zi_unit"]
    # now we have our initial conditions, thus we filter the new samples with those
    # initial conditions and we overwrite the initial conditions with the state of the
    # filter at the end of the filtering process, preparing the next iteration.
    new_samples, filter["zi"] = sosfilt(filter["sos"], new_samples, zi=filter["zi"])

    # update the buffer, using the rolling system details previously.
    buffer = np.roll(buffer, -new_samples.size)
    buffer[-new_samples.size :] = new_samples

This loop contains all of the acquisition logic used in `mne_lsl.stream.StreamLSL`.

### Detecting new peak and delivering sounds

Now that we have a filtered buffer, we look for peaks. It boils down to extending the
previous loop to:

In [ ]:
# Pseudo-code, do not run this cell

# construct filter
from scipy.signal import sosfilt, sosfilt_zi

filter = create_filter(...)  # create second order sections (sos)  # noqa: A001
filter["zi_unit"] = sosfilt_zi(filter["sos"])  # create initial conditions
filter["zi"] = (
    None  # initial conditions for the filter that will be updated in the first iteration of the loop  # noqa: E501
)

# create buffer
import numpy as np  # noqa: E402

buffer = np.zeros(...)
timestamps_buffer = np.zeros(...)

# online acquisition loop
while True:
    # query the network for new samples, we might get some or we might get none
    new_samples, new_samples_timestamps = inlet.pull_chunk()  # noqa: F821
    if new_samples_timestamps.size == 0:
        # no new samples, thus we 'continue', i.e. we go back to the beginning of the
        # loop and query the network again (without delay) for new samples
        continue

    # process the new samples, in this case apply the filter. We need to know the filter
    # state 'zi', which is either the state of the N-1 iteration, or the initial state
    # estimated as the steady-state response of the filter to a step input.
    if filter["zi"] is None:
        # first iteration, estimate using a step input equal to the mean of the signal
        # which is realistic for EEG signal subject to an important DC offset.
        filter["zi"] = np.mean(new_samples) * filter["zi_unit"]
    # now we have our initial conditions, thus we filter the new samples with those
    # initial conditions and we overwrite the initial conditions with the state of the
    # filter at the end of the filtering process, preparing the next iteration.
    new_samples, filter["zi"] = sosfilt(filter["sos"], new_samples, zi=filter["zi"])

    # update the buffer, using the rolling system details previously.
    buffer = np.roll(buffer, -new_samples.size)
    buffer[-new_samples.size :] = new_samples
    timestamps_buffer = np.roll(timestamps_buffer, -new_samples_timestamps.size)
    timestamps_buffer[-new_samples_timestamps.size :] = new_samples_timestamps

    # detrend the signal
    z = np.polyfit(timestamps_buffer, buffer, 1)
    data = buffer - (z[0] * timestamps_buffer + z[1])

    # detect all the peaks in the buffer, this function tells you at which sample in the
    # buffer a peak is present, e.g. peaks = np.array([100, 1800, 3500]) means that I
    # have 3 peaks at the index 100, 1800, and 3500 of the buffer.
    peaks, _ = find_peaks(data, height=..., prominence=...)

    # triage the peaks to figure out if any of those peaks is new, because either it was
    # already detected and processed in the past, or it was detected for the first time
    # and is thus a new, very recent, peak.
    # if new_peak is None, the peak was already returned in the past. If the peak was
    # not returned in the past, returns the index of the new peak in the buffer, e.g.
    # new_peak = 1000 means that the new peak is at the index 1000 of the buffer.
    new_peak = triage_peaks(peaks)  # noqa

    if new_peak is None:
        # we don't have any new peak to play with, let's go back to the beginning of the
        # loop and query the network for new samples.
        continue

    # process the new peak, first let's figure out how late we are:
    now = local_clock()  # current time on the same timescale as the timestamps buffer
    late_by = now - timestamps_buffer[new_peak]

    # now let's plan the sound in the future (I'm a bit simplifying here because we
    # have 2 different clock base to handle, LSL and PsychtoolBox, but the idea is to
    # plan the sound 250 ms after the peak by correcting based on 'late_by').
    sound.play(when=timestamps_buffer[new_peak] + 0.25)
    # now we wait until the sound is going to be played, again we correct by how 'late'
    # we were im the detection.
    sleep(...)
    trigger.signal(...)  # and now we deliver the trigger, simultaneously with the sound

And that's the general structure of the online loop used here for both synchronous conditions. Now, the problem we observe between online detection and offline replication, is that the filtered buffer on which `find_peaks` is run differ! Exactly as shown in the beginning of the notebook where 2 different filters yields 2 slightly different waveforms with different local maxima.